<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_accessibility_open_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')#.drop(['Addresses'], axis=1)
data = data.loc[data['funding_yes'] == 1]
data.head(2)

,Unnamed: 0,wos_ID,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
1,1,wos:000921279800001,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
2,2,wos:000909001200001,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0


In [ ]:
data.count().head(10)

Unnamed: 0                 577
wos_ID                     577
funding_yes                577
open_access_yes            577
Funding Name Preferred     575
europe and central asia    125
north america              431
africa                       2
sub-saharan africa          20
east asia and pacific       78
dtype: int64

In [ ]:
data['funding_yes'].value_counts()

1    577
Name: funding_yes, dtype: int64

In [ ]:
data.groupby(['open_access_yes']).count()

,Unnamed: 0,wos_ID,funding_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
open_access_yes,,,,,,,,,,,,,,,,,,,,,
0,78,78,78,78,21,45,2,5,14,1,...,0,0,0,0,78,0,68,27,75,75
1,499,499,499,497,104,386,0,15,64,9,...,1,1,1,1,499,499,479,209,485,498


# The accessibility of Marburg Virus Diseases publications (open access)

## proportion all

In [ ]:
countries_oa_list_all = data.reset_index().groupby(['open_access_yes']).count().T
countries_oa_list_all.rename(columns = {0:'all_is_no_oa',
                              1:'all_is_oa'}, inplace = True)

countries_oa_list_all['all_total'] = countries_oa_list_all['all_is_no_oa'] + countries_oa_list_all['all_is_oa']

for col in countries_oa_list_all.columns:
    countries_oa_list_all['%'+col] = countries_oa_list_all[col]/countries_oa_list_all['all_total']*100

countries_oa_list_all['%_diff(%all_is_oa-%all_is_no_oa)'] = countries_oa_list_all['%all_is_oa'] - countries_oa_list_all['%all_is_no_oa']

# save data
countries_oa_list_all.to_csv(f'../TABLES/region_countries_org_all_oa_list.csv')

countries_oa_list_all

open_access_yes,all_is_no_oa,all_is_oa,all_total,%all_is_no_oa,%all_is_oa,%all_total,%_diff(%all_is_oa-%all_is_no_oa)
index,78,499,577,13.518198,86.481802,100.0,72.963605
Unnamed: 0,78,499,577,13.518198,86.481802,100.0,72.963605
wos_ID,78,499,577,13.518198,86.481802,100.0,72.963605
funding_yes,78,499,577,13.518198,86.481802,100.0,72.963605
Funding Name Preferred,78,497,575,13.565217,86.434783,100.0,72.869565
...,...,...,...,...,...,...,...
Open Access Designations,0,499,499,0.000000,100.000000,100.0,100.000000
"Times Cited, All Databases",68,479,547,12.431444,87.568556,100.0,75.137112
180 Day Usage Count,27,209,236,11.440678,88.559322,100.0,77.118644
Since 2013 Usage Count,75,485,560,13.392857,86.607143,100.0,73.214286


In [ ]:
xxxxx

NameError: name 'xxxxx' is not defined

In [ ]:
countries_oa_list_all[['all_is_no_oa','all_is_oa','all_total']].describe()

### significance

In [ ]:
all_OA = data['funding_yes'].loc[data['open_access_yes'] == 1].to_list()
all_no_OA = data['funding_yes'].loc[data['open_access_yes'] == 0].to_list()

test_sig(all_OA, all_no_OA)

## proportion fund

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.shape

In [ ]:
countries_oa_list_fund = data_fund.reset_index().groupby(['open_access_yes']).count().T
countries_oa_list_fund.rename(columns = {0:'fund_is_no_oa',
                              1:'fund_is_oa'}, inplace = True)

countries_oa_list_fund['fund_total'] = countries_oa_list_fund['fund_is_no_oa'] + countries_oa_list_fund['fund_is_oa']

for col in countries_oa_list_fund.columns:
    countries_oa_list_fund['%'+col] = countries_oa_list_fund[col]/countries_oa_list_fund['fund_total']*100

countries_oa_list_fund['%_diff(%fund_is_oa-%fund_is_no_oa)'] = countries_oa_list_fund['%fund_is_oa'] - countries_oa_list_fund['%fund_is_no_oa']

# save data
# countries_oa_list_fund.to_csv(f'../TABLES/countries_fund_oa_list.csv')

countries_oa_list_fund

## proportion no fund

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund.head(2)

In [ ]:
countries_oa_list_no_fund = data_no_fund.reset_index().groupby(['open_access_yes']).count().T
countries_oa_list_no_fund.rename(columns = {0:'no_fund_is_no_oa',
                              1:'no_fund_is_oa'}, inplace = True)

countries_oa_list_no_fund['no_fund_total'] = countries_oa_list_no_fund['no_fund_is_no_oa'] + countries_oa_list_no_fund['no_fund_is_oa']

for col in countries_oa_list_no_fund.columns:
    countries_oa_list_no_fund['%'+col] = countries_oa_list_no_fund[col]/countries_oa_list_no_fund['no_fund_total']*100

countries_oa_list_no_fund['%_diff(%no_fund_is_oa-%no_fund_is_no_oa)'] = countries_oa_list_no_fund['%no_fund_is_oa'] - countries_oa_list_no_fund['%no_fund_is_no_oa']

# save data
# countries_oa_list_no_fund.to_csv(f'../TABLES/countries_fund_no_oa_list.csv')

countries_oa_list_no_fund

## concat all dataframe and save

In [ ]:
# concatenate dataframes on index
concat_countries_oa_list = pd.concat([countries_oa_list_all,
                                        countries_oa_list_fund,
                                        countries_oa_list_no_fund],
                                       axis=1)
# save data
concat_countries_oa_list.to_csv(f'../TABLES/concat_countries_oa_list.csv')
concat_countries_oa_list